# Constructing a GarNet file

The goal here is to construct a file in which all of the gene and transcription factor names exist in the same namespace. We use mygene.info's API to map all the gene names to a common namespace. It isn't clear that they have the most "canonical" namespace but at present they correlate best with genecards, and seems to be more consistent than all other namespaces I know of. 

In [2]:
%pylab inline
import sys
import os
import pickle
import sqlite3
import numpy as np
import pandas as pd
from intervaltree import IntervalTree


known_genes_file = '../data/ucsc_hg19_knownGenes.tsv'
kgXref_file = '../data/ucsc_hg19_kgXref.tsv'
motifs_file = '../example_data/motifmap.normalized.tsv'

Populating the interactive namespace from numpy and matplotlib


# Part I: Constructing a reference file

We use UCSC's [known genes](http://genome.ucsc.edu/cgi-bin/hgTables?hgsid=590772967_aCXvu74nAfyUAYeUksjLuUk1eBz3&clade=mammal&org=Human&db=hg19&hgta_group=genes&hgta_track=refGene&hgta_table=refGene&hgta_regionType=genome&position=chr21%3A33031597-33041570&hgta_outputType=primaryTable&hgta_outFileName=) and [Cross Reference (kgXref) file](http://genome.ucsc.edu/cgi-bin/hgTables?hgsid=590772967_aCXvu74nAfyUAYeUksjLuUk1eBz3&clade=mammal&org=Human&db=hg19&hgta_group=genes&hgta_track=refGene&hgta_table=kgXref&hgta_regionType=genome&position=chr21%3A33%2C031%2C597-33%2C041%2C570&hgta_outputType=primaryTable&hgta_outFileName=) as our foundation. 

In [3]:
def parse_known_genes_file(known_genes_file, kgXref_file):
    """
    Parse the RefSeq known genes file into a pandas dataframe

    The known genes file format is the following:
    http://hgdownload.cse.ucsc.edu/goldenPath/hg19/database/knownGene.sql

    Arguments:
        known_genes_file (string or FILE): file procured from RefSeq with full list of genes in genome
        kgXref_file (string or FILE): additional "Cross Reference" file with more details on those genes

    Returns:
        dataframe: known genes dataframe
    """

    known_genes_fieldnames = ["name","chrom","strand","txStart","txEnd","cdsStart","cdsEnd","exonCount","exonStarts","exonEnds","proteinID","alignID"]

    known_genes_dataframe = pd.read_csv(known_genes_file, delimiter='\t', names=known_genes_fieldnames)

    known_genes_dataframe.rename(index=str, columns={"txStart":"geneStart", "txEnd":"geneEnd", "name":"geneName","strand":"geneStrand"}, inplace=True)

    if kgXref_file:

        kgXref_fieldnames = ["kgID","mRNA","spID","spDisplayID","geneSymbol","refseq","protAcc","description"]
        kgXref_dataframe = pd.read_csv(kgXref_file, delimiter='\t', names=kgXref_fieldnames)

        known_genes_dataframe = known_genes_dataframe.merge(kgXref_dataframe, left_on='geneName', right_on='kgID', how='left')
        known_genes_dataframe.rename(index=str, columns={"geneName":"ucID", "geneSymbol":"geneName"}, inplace=True)

    return known_genes_dataframe

In [4]:
reference = parse_known_genes_file(known_genes_file, kgXref_file)
reference.head()

,ucID,chrom,geneStrand,geneStart,geneEnd,cdsStart,cdsEnd,exonCount,exonStarts,exonEnds,proteinID,alignID,kgID,mRNA,spID,spDisplayID,geneName,refseq,protAcc,description
0,uc001aaa.3,chr1,+,11873,14409,11873,11873,3,"11873,12612,13220,","12227,12721,14409,",NaN,uc001aaa.3,uc001aaa.3,NR_046018,NaN,NaN,DDX11L1,NR_046018,NaN,Homo sapiens DEAD/H (Asp-Glu-Ala-Asp/His) box ...
1,uc010nxr.1,chr1,+,11873,14409,11873,11873,3,"11873,12645,13220,","12227,12697,14409,",NaN,uc010nxr.1,uc010nxr.1,AM992878,NaN,NaN,DDX11L1,NaN,NaN,Homo sapiens DEAD/H (Asp-Glu-Ala-Asp/His) box ...
2,uc010nxq.1,chr1,+,11873,14409,12189,13639,3,"11873,12594,13402,","12227,12721,14409,",B7ZGX9,uc010nxq.1,uc010nxq.1,AM992880,B7ZGX9,B7ZGX9_HUMAN,DDX11L1,NaN,NaN,Homo sapiens DEAD/H (Asp-Glu-Ala-Asp/His) box ...
3,uc009vis.3,chr1,-,14361,16765,14361,14361,4,"14361,14969,15795,16606,","14829,15038,15942,16765,",NaN,uc009vis.3,uc009vis.3,BC047449,NaN,NaN,WASH7P,NaN,NaN,Homo sapiens WAS protein family homolog 7 pseu...
4,uc009vjc.1,chr1,-,16857,17751,16857,16857,2,"16857,17232,","17055,17751,",NaN,uc009vjc.1,uc009vjc.1,AK291582,NaN,NaN,WASH7P,NaN,NaN,Homo sapiens WAS protein family homolog 7 pseu...


In [5]:
reference = reference[['geneName', 'ucID', 'chrom', 'geneStrand', 'geneStart', 'geneEnd']]
reference.head()

,geneName,ucID,chrom,geneStrand,geneStart,geneEnd
0,DDX11L1,uc001aaa.3,chr1,+,11873,14409
1,DDX11L1,uc010nxr.1,chr1,+,11873,14409
2,DDX11L1,uc010nxq.1,chr1,+,11873,14409
3,WASH7P,uc009vis.3,chr1,-,14361,16765
4,WASH7P,uc009vjc.1,chr1,-,16857,17751


In [6]:
reference[reference.geneName.isnull()]

,geneName,ucID,chrom,geneStrand,geneStart,geneEnd


### ...Amazing!

In [7]:
import mygene
mg = mygene.MyGeneInfo()
df = mg.querymany(np.unique(reference.geneName.values).tolist(), scopes=['symbol', 'name', 'alias'], fields=["HGNC", "symbol"], species="human", as_dataframe=True, returnall=True)
df

querying 1-1000...done.
querying 1001-2000...done.
querying 2001-3000...done.
querying 3001-4000...done.
querying 4001-5000...done.
querying 5001-6000...done.
querying 6001-7000...done.
querying 7001-8000...done.
querying 8001-9000...done.
querying 9001-10000...done.
querying 10001-11000...done.
querying 11001-12000...done.
querying 12001-13000...done.
querying 13001-14000...done.
querying 14001-15000...done.
querying 15001-16000...done.
querying 16001-17000...done.
querying 17001-18000...done.
querying 18001-19000...done.
querying 19001-20000...done.
querying 20001-21000...done.
querying 21001-22000...done.
querying 22001-23000...done.
querying 23001-24000...done.
querying 24001-25000...done.
querying 25001-26000...done.
querying 26001-27000...done.
querying 27001-28000...done.
querying 28001-28517...done.
Finished.
4439 input query terms found dup hits:
	[('5S_rRNA', 10), ('5_8S_rRNA', 6), ('7SK', 10), ('A1BG', 2), ('A2M', 5), ('A2ML1', 5), ('AADACL2', 
5269 input query terms found n

{'dup': [('5S_rRNA', 10),
  ('5_8S_rRNA', 6),
  ('7SK', 10),
  ('A1BG', 2),
  ('A2M', 5),
  ('A2ML1', 5),
  ('AADACL2', 4),
  ('AARSD1', 2),
  ('AATK', 2),
  ('ABCA9', 2),
  ('ABCC2', 2),
  ('ABCC5', 2),
  ('ABCC6P1', 2),
  ('ABHD11', 3),
  ('ABHD14A', 2),
  ('ABHD15', 3),
  ('ABHD17A', 3),
  ('ABO', 2),
  ('ABP1', 2),
  ('ABRA', 2),
  ('ACAD11', 3),
  ('ACAP2', 3),
  ('ACAT1', 2),
  ('ACAT2', 2),
  ('ACBD3', 3),
  ('ACBD7', 2),
  ('ACOXL', 2),
  ('ACP1', 2),
  ('ACTA2', 3),
  ('ACTB', 2),
  ('ACTG1P4', 2),
  ('ACTN1', 2),
  ('ACTR3', 7),
  ('ACTR3B', 10),
  ('ACTR3BP2', 2),
  ('ACTR3BP5', 2),
  ('ACTR6', 3),
  ('ACVR2B', 2),
  ('ACY1', 2),
  ('AD', 10),
  ('ADA', 3),
  ('ADAM18', 2),
  ('ADAM1A', 2),
  ('ADAM5', 2),
  ('ADAMTS19', 2),
  ('ADAMTS7', 8),
  ('ADAMTS9', 4),
  ('ADAMTS9-AS2', 2),
  ('ADAMTSL4', 2),
  ('ADARB2', 2),
  ('ADC', 2),
  ('ADCY3', 2),
  ('ADD3', 3),
  ('ADH4', 2),
  ('ADIPOQ', 9),
  ('ADIRF', 3),
  ('ADNP', 3),
  ('ADORA2A', 4),
  ('ADPGK', 2),
  ('ADRA1A', 2),
 

In [8]:
dup = df['dup']
missing = df['missing']
df = df['out']
df = df[["HGNC", "symbol"]].dropna()
df = df.reset_index().drop_duplicates(subset='query', keep='first').set_index('query').rename_axis(None)
df

,HGNC,symbol
6M1-18,8176,OR11A1
7M1-2,8246,OR2F1
A1BG,5,A1BG
A1BG-AS1,37133,A1BG-AS1
A1CF,24086,A1CF
A2M,7,A2M
A2M-AS1,27057,A2M-AS1
A2ML1,23336,A2ML1
A2MP1,8,A2MP1
A3GALT2,30005,A3GALT2


In [9]:
len(df[df.index != df.symbol])

1067

In [10]:
missing

['AB000466',
 'AB007962',
 'AB059369',
 'AB062081',
 'AB062083',
 'AB073649',
 'AB074160',
 'AB074162',
 'AB074166',
 'AB074188',
 'AB075489',
 'AB075492',
 'AB209061',
 'AB209185',
 'AB209315',
 'AB209621',
 'AB231702',
 'AB231703',
 'AB231705',
 'AB231710',
 'AB231711',
 'AB231721',
 'AB231722',
 'AB231723',
 'AB231724',
 'AB231729',
 'AB231731',
 'AB231739',
 'AB231741',
 'AB231742',
 'AB231761',
 'AB231779',
 'AB231784',
 'AB240015',
 'AB372727',
 'AB429224',
 'AB488780',
 'AB586698',
 '1',
 'ADV21S1A1N',
 'AF007147',
 'AF020763',
 'AF035281',
 'AF047486',
 'AF055024',
 'AF063596',
 'AF070569',
 'AF070581',
 'AF072097',
 'AF075036',
 'AF075112',
 'AF079515',
 'AF085962',
 'AF085995',
 'AF086102',
 'AF086125',
 'AF086126',
 'AF086132',
 'AF086154',
 'AF086165',
 'AF086184',
 'AF086203',
 'AF086219',
 'AF086258',
 'AF086285',
 'AF086288',
 'AF086294',
 'AF086303',
 'AF086346',
 'AF086351',
 'AF086476',
 'AF088041',
 'AF090102',
 'AF090939',
 'AF116693',
 'AF119915',
 'AF131837',
 'AF

In [11]:
reference = reference.merge(df, how='left', left_on='geneName', right_index=True)
reference.head()

,geneName,ucID,chrom,geneStrand,geneStart,geneEnd,HGNC,symbol
0,DDX11L1,uc001aaa.3,chr1,+,11873,14409,37102,DDX11L1
1,DDX11L1,uc010nxr.1,chr1,+,11873,14409,37102,DDX11L1
2,DDX11L1,uc010nxq.1,chr1,+,11873,14409,37102,DDX11L1
3,WASH7P,uc009vis.3,chr1,-,14361,16765,38034,WASH7P
4,WASH7P,uc009vjc.1,chr1,-,16857,17751,38034,WASH7P


In [12]:
reference.symbol = reference.symbol.fillna(reference.geneName)

In [13]:
del reference['geneName']
reference.rename(columns={'symbol': 'geneName'}, inplace=True)
reference.head()

,ucID,chrom,geneStrand,geneStart,geneEnd,HGNC,geneName
0,uc001aaa.3,chr1,+,11873,14409,37102,DDX11L1
1,uc010nxr.1,chr1,+,11873,14409,37102,DDX11L1
2,uc010nxq.1,chr1,+,11873,14409,37102,DDX11L1
3,uc009vis.3,chr1,-,14361,16765,38034,WASH7P
4,uc009vjc.1,chr1,-,16857,17751,38034,WASH7P


In [14]:
reference = reference[['geneName', 'chrom', 'geneStrand', 'geneStart', 'geneEnd']]
reference.head()

,geneName,chrom,geneStrand,geneStart,geneEnd
0,DDX11L1,chr1,+,11873,14409
1,DDX11L1,chr1,+,11873,14409
2,DDX11L1,chr1,+,11873,14409
3,WASH7P,chr1,-,14361,16765
4,WASH7P,chr1,-,16857,17751


In [15]:
(len(reference), len(reference.drop_duplicates()))

(82960, 60840)

In [16]:
reference.drop_duplicates(inplace=True)

In [17]:
reference.to_csv('../example_data/reference.normalized.tsv', sep='\t', index=False, header=True)

# Part II: Constructing a motifs file

Important! I've already normalized the MotifMap file, so I won't do it here. If you're running this again, make sure to normalize first, or use the normalized file I created!

In [18]:
def parse_motifs_file(motifs_file):
    """
    Parse the MotifMap BED file listing Transcription Factor Binding Motifs in the genome

    Arguments:
        motifs_file (string or FILE): file procured from MotifMap with full list of TF binding sites in the genome

    Returns:
        dataframe: motif dataframe
    """

    motif_fieldnames = ["ZScore","FDR_lower","name","orientation","chrom","LOD","strand","start","realhits","cid","FDR","NLOD","BBLS","stop","medianhits","accession","FDR_upper","BLS","stdevhits"]

    motif_dataframe = pd.read_csv(motifs_file, delimiter='\t', names=motif_fieldnames)

    motif_dataframe.rename(index=str, columns={"start":"motifStart", "stop":"motifEnd", "FDR":"motifScore", "strand":"motifStrand", "name":"motifName"}, inplace=True)

    return motif_dataframe


In [26]:
motifs = parse_motifs_file(motifs_file)
motifs.head()

,ZScore,FDR_lower,motifName,orientation,chrom,LOD,motifStrand,motifStart,realhits,cid,motifScore,NLOD,BBLS,motifEnd,medianhits,accession,FDR_upper,BLS,stdevhits
0,6.434480,0.0,RFX1,1,chr17,29.355633,+,58239126,1,1,0.0,0.992106,1.763390,58239144,0.0,LM1_RFX1,0.0,3.349127,0.0
1,6.430004,0.0,RFX1,-1,chr17,29.320824,-,58239126,2,2,0.0,0.991646,1.763377,58239144,0.0,LM1_RFX1,0.0,3.349127,0.0
2,6.368495,0.0,RFX1,-1,chr5,28.842535,-,63404689,3,3,0.0,0.985325,0.635808,63404707,0.0,LM1_RFX1,0.0,2.725784,0.0
3,6.364415,0.0,RFX1,1,chr5,28.810813,+,63404689,4,4,0.0,0.984906,0.635796,63404707,0.0,LM1_RFX1,0.0,2.725784,0.0
4,6.353193,0.0,RFX1,1,chr7,28.723547,+,128800229,2,5,0.0,0.983752,1.815382,128800247,0.0,LM1_RFX1,0.0,4.027027,0.0


In [27]:
motifs = motifs[['motifName', 'chrom', 'motifStrand', 'motifStart', 'motifEnd', 'motifScore']]
motifs.head()

,motifName,chrom,motifStrand,motifStart,motifEnd,motifScore
0,RFX1,chr17,+,58239126,58239144,0.0
1,RFX1,chr17,-,58239126,58239144,0.0
2,RFX1,chr5,-,63404689,63404707,0.0
3,RFX1,chr5,+,63404689,63404707,0.0
4,RFX1,chr7,+,128800229,128800247,0.0


In [28]:
(len(motifs), len(motifs.drop_duplicates()))

(17309990, 17301513)

In [29]:
motifs = motifs.drop_duplicates()

In [30]:
motifs.to_csv('../example_data/motifmap.normalized.cleaned.tsv', sep='\t', index=False, header=True)

# Part III: Merging genes and motifs

In [2]:
reference = pd.read_csv('../example_data/reference.normalized.tsv', sep='\t')
motifs = pd.read_csv('../example_data/motifmap.normalized.cleaned.tsv', sep='\t')

In [3]:
reference.head()

,geneName,chrom,geneStrand,geneStart,geneEnd
0,DDX11L1,chr1,+,11873,14409
1,WASH7P,chr1,-,14361,16765
2,WASH7P,chr1,-,16857,17751
3,WASH7P,chr1,-,15795,18061
4,WASH7P,chr1,-,14361,19759


In [4]:
motifs.head()

,motifName,chrom,motifStrand,motifStart,motifEnd,motifScore
0,RFX1,chr17,+,58239126,58239144,0.0
1,RFX1,chr17,-,58239126,58239144,0.0
2,RFX1,chr5,-,63404689,63404707,0.0
3,RFX1,chr5,+,63404689,63404707,0.0
4,RFX1,chr7,+,128800229,128800247,0.0


In [3]:
def group_by_chromosome(dataframe):
	"""
	Arguments:
		dataframe (dataframe): Must be a dataframe with a chrom column

	Returns:
		dict: dictionary of chromosome names (e.g. 'chr1') to dataframes
	"""

	return dict(list(dataframe.groupby('chrom')))


def IntervalTree_from_reference(reference, options):
	"""
	Arguments:
		reference (dataframe): Must be a dataframe with `strand`, `geneStart`, and `geneEnd` columns
		options (dict): {"upstream_window": int, "downstream_window": int, "tss": bool}

	Returns:
		IntervalTree: of genes from the reference
	"""

	upstream_window = options.get('upstream_window') or 2000
	downstream_window = options.get('downstream_window') or 2000
	window_ends_downstream_from_transcription_start_site_instead_of_transcription_end_site = options.get('tss') or False

	if window_ends_downstream_from_transcription_start_site_instead_of_transcription_end_site:
		starts = reference.apply(lambda x: x.geneStart - upstream_window if x.strand == '+' else x.geneEnd - upstream_window, axis=1)
		ends = reference.apply(lambda x: x.geneStart + downstream_window if x.strand == '+' else x.geneEnd + downstream_window, axis=1)

	else:
		starts = reference.apply(lambda x: x.geneStart - upstream_window, axis=1)
		ends = reference.apply(lambda x: x.geneEnd + downstream_window, axis=1)

	intervals = zip(starts.values, ends.values, reference.to_dict(orient='records'))

	tree = IntervalTree.from_tuples(intervals)

	return tree


def IntervalTree_from_motifs(motifs):
	"""
	Arguments:
		motifs (dataframe): Must be a dataframe with motifStart and motifEnd columns

	Returns:
		IntervalTree: of motifs
	"""

	intervals = zip(motifs.motifStart.values, motifs.motifEnd.values, motifs.to_dict(orient='records'))

	tree = IntervalTree.from_tuples(intervals)

	return tree

In [10]:
reference = group_by_chromosome(reference)
motifs = group_by_chromosome(motifs)

AttributeError: 'dict' object has no attribute 'groupby'

In [15]:
options = {'upstream_window': 10000, 'downstream_window': 10000, 'tss': False}
reference = {chrom: IntervalTree_from_reference(genes, options) for chrom, genes in reference.items()}
motifs = {chrom: IntervalTree_from_motifs(chromosome_motifs) for chrom, chromosome_motifs in motifs.items()}

In [4]:
def intersection_of_dict_of_intervaltree(A, B):
	"""
	Arguments:
		A (dict): is a dictionary of {chrom (str): IntervalTree}
		B (dict): is a dictionary of {chrom (str): IntervalTree}

	Returns:
		dict: {keys shared between A and B: {intervals in A: [list of overlapping intervals in B]} }
	"""

	# Keys are chromosomes. We only want to look through chromosomes where there is potential for overlap
	common_keys = set(A.keys()).intersection( set(B.keys()) )

	# In general, the below operation isn't perfectly elegant, due to the double-for:
	#   `for key in common_keys for a in A[key]`
	# The reason we use a double-for here is because we want to do one level of "flattening"
	# but we don't want to do it as a pre-processing or post-processing step. Specifically,
	# we're passed dictionaries of chrom: IntervalTree, and we'd like to return a single data
	# structure for the entire genome, not one per chromosome. The below can be read as:
	#
	#	for chromosome in chromosomes_found_in_both_datastructures:
	#		for a_interval in A[chromosome]
	#			for b_interval in B[chromosome].search(A_interval)
	#				Map a_interval to b_interval
	#
	# which can be expressed concisely in the following line of code (which is also maximally efficient)
	intersection = [(a.data, b.data) for key in common_keys for a in A[key] for b in B[key].search(a)]

	return intersection

In [19]:
motifs_with_associated_genes = intersection_of_dict_of_intervaltree(motifs, reference)

In [20]:
motifs_and_genes = [{**motif, **gene} for motif, gene in motifs_with_associated_genes]
motifs_and_genes = pd.DataFrame.from_records(motifs_and_genes)
motifs_and_genes.head()

,chrom,geneEnd,geneName,geneStart,geneStrand,motifEnd,motifName,motifScore,motifStart,motifStrand
0,chr10,114927436,TCF7L2,114799762,+,114808426,ZNF354C,0.577708,114808420,-
1,chr10,114927436,TCF7L2,114710881,+,114808426,ZNF354C,0.577708,114808420,-
2,chr10,114927436,TCF7L2,114710008,+,114808426,ZNF354C,0.577708,114808420,-
3,chr10,1779670,ADARB2,1223252,-,1657270,DBP,0.896767,1657263,+
4,chr10,70454239,TET1,70320116,+,70363025,SOX10,0.019214,70363018,+


In [21]:
motifs_and_genes = motifs_and_genes[['chrom','motifName','motifStrand','motifStart','motifEnd','motifScore','geneName','geneStart','geneEnd','geneStrand']]
motifs_and_genes

,chrom,motifName,motifStrand,motifStart,motifEnd,motifScore,geneName,geneStart,geneEnd,geneStrand
0,chr10,ZNF354C,-,114808420,114808426,0.577708,TCF7L2,114799762,114927436,+
1,chr10,ZNF354C,-,114808420,114808426,0.577708,TCF7L2,114710881,114927436,+
2,chr10,ZNF354C,-,114808420,114808426,0.577708,TCF7L2,114710008,114927436,+
3,chr10,DBP,+,1657263,1657270,0.896767,ADARB2,1223252,1779670,-
4,chr10,SOX10,+,70363018,70363025,0.019214,TET1,70320116,70454239,+
5,chr10,IRF1,+,119869461,119869468,0.512229,CASC2,119806331,119859647,+
6,chr10,IRF1,+,119869461,119869468,0.512229,CASC2,119806331,119969665,+
7,chr10,GATA2,+,116397441,116397446,1.000000,ABLIM1,116190868,116444414,-
8,chr10,GATA2,+,116397441,116397446,1.000000,ABLIM1,116200775,116392100,-
9,chr10,GATA2,+,116397441,116397446,1.000000,ABLIM1,116190868,116418058,-


In [22]:
motifs_and_genes['motif_to_gene_distance'] = motifs_and_genes['motifStart'] - motifs_and_genes['geneStart']
motifs_and_genes.head()

,chrom,motifName,motifStrand,motifStart,motifEnd,motifScore,geneName,geneStart,geneEnd,geneStrand,motif_to_gene_distance
0,chr10,ZNF354C,-,114808420,114808426,0.577708,TCF7L2,114799762,114927436,+,8658
1,chr10,ZNF354C,-,114808420,114808426,0.577708,TCF7L2,114710881,114927436,+,97539
2,chr10,ZNF354C,-,114808420,114808426,0.577708,TCF7L2,114710008,114927436,+,98412
3,chr10,DBP,+,1657263,1657270,0.896767,ADARB2,1223252,1779670,-,434011
4,chr10,SOX10,+,70363018,70363025,0.019214,TET1,70320116,70454239,+,42902


In [23]:
motifs_and_genes.to_csv('../example_data/intersection.tsv', sep='\t', header=True, index=False)

# Part IV: Building and exporting IntervalTrees: constructing the "GarNet File"

In [5]:
motifs_and_genes = pd.read_csv('../example_data/intersection.tsv', sep='\t')

In [6]:
motifs_and_genes = group_by_chromosome(motifs_and_genes)

In [7]:
motifs_and_genes = {chrom: IntervalTree_from_motifs(chromosome_motifs_and_genes) for chrom, chromosome_motifs_and_genes in motifs_and_genes.items()}

In [8]:
def save_as_pickled_object(obj, directory, filename):
	"""
	This is a defensive way to write pickle.write, allowing for very large files on all platforms
	"""
	filepath = os.path.join(directory, filename)
	max_bytes = 2**31 - 1
	bytes_out = pickle.dumps(obj)
	n_bytes = sys.getsizeof(bytes_out)
	with open(filepath, 'wb') as f_out:
		for idx in range(0, n_bytes, max_bytes):
			f_out.write(bytes_out[idx:idx+max_bytes])

In [9]:
save_as_pickled_object(motifs_and_genes, '../example_data/', 'garnet.pickle')